In [ ]:
# Read the file and parse the events
filename = "EPTdata-Lecture-Lefeber-1.txt"

# Load data
events = []
with open(filename, 'r') as f:
    for line in f:
        parts = line.strip().split('\t')
        time = int(parts[0])
        lot = parts[1]
        event_type = parts[2]
        # Extract lot number from lot name
        lot_number = int(lot.replace('lot', ''))
        events.append((time, event_type, lot_number))

# EPT Algorithm

def detOvert(xs, i):
    """
    Determine the overtaking status of lots in the system.
    :param xs: List of tuples (arrival_number, aw) for lots in the system
    :param i: Arrival number of the departing lot
    :return: Tuple containing the updated list of lots, number of overtaken lots, and WIP level at arrival
    """
    ys = []  # List to store lots that remain after processing
    overtaken_count = 0  # Number of overtaken lots
    aw = 0  # WIP level at the time of the departing lot's arrival

    for idx, (j, wip) in enumerate(xs):
        if j < i:
            ys.append((j, wip))  # Keep lots that arrived before the departing lot
        elif j == i:
            aw = wip  # WIP level when the departing lot arrived
            # Count all lots that arrived after this lot but departed earlier
            overtaken_count = len(xs[:idx])  # Lots before this index are overtaken
            break

    # Remove the departing lot from the list
    ys.extend(xs[idx + 1:])

    return ys, overtaken_count, aw

xs = []  # list of (arrival_number, aw)
s = None  # EPT start time
sw = None  # WIP-level after EPT start

# Outputs
EPT_results = []  # (EPT_duration, WIP-level)
Overtaking_results = []  # (number_overtaken, aw)

for (tau, ev, i) in events:
    if ev == 'A':
        if len(xs) == 0:
            s, sw = tau, 1
        xs.append((i, len(xs)))
    elif ev == 'D':
        # End current EPT
        EPT_duration = tau - s
        EPT_results.append((EPT_duration, sw))
        
        xs, k, aw = detOvert(xs, i)
        Overtaking_results.append((k, aw, i))

        if len(xs) > 0:
            s, sw = tau, len(xs)

# Display Results
print("EPT Durations and WIP-levels:")
for duration, wip in EPT_results:
    print(f"EPT Duration: {duration}, WIP-level: {wip}")

print("\nOvertaking Details:")
for overtaken, aw, i in Overtaking_results:
    print(f"Lot: {i} Number of overtaken lots: {overtaken}, WIP before arrival: {aw}")

# Calculate and display mean EPT duration
mean_EPT = sum(duration for duration, _ in EPT_results) / len(EPT_results)
print(f"\nMean EPT Duration: {mean_EPT:.2f}")


EPT Durations and WIP-levels:
EPT Duration: 4, WIP-level: 1
EPT Duration: 2, WIP-level: 3
EPT Duration: 1, WIP-level: 3
EPT Duration: 4, WIP-level: 2
EPT Duration: 1, WIP-level: 3
EPT Duration: 3, WIP-level: 2
EPT Duration: 0, WIP-level: 1

Overtaking Details:
Lot 1: Number of overtaken lots: 0, WIP before arrival: 0
Lot 2: Number of overtaken lots: 0, WIP before arrival: 1
Lot 3: Number of overtaken lots: 0, WIP before arrival: 2
Lot 5: Number of overtaken lots: 1, WIP before arrival: 3
Lot 4: Number of overtaken lots: 0, WIP before arrival: 3
Lot 6: Number of overtaken lots: 0, WIP before arrival: 2
Lot 7: Number of overtaken lots: 0, WIP before arrival: 3

Mean EPT Duration: 2.14
